In [ ]:
import os
if not os.path.exists("covid"):
  !git clone https://github.com/earwole1/covid covid

In [ ]:
import sys
sys.path.append("./covid")
import covid
c = covid.Covid()

In [ ]:
_ = c.plot_state("Florida", "cases")

In [ ]:
_ = [c.plot_state("Ohio", x) for x in ["cases", "deaths", 'hospitalizedCurrently']]

In [ ]:
_ = c.plot_all_by_date()

In [ ]:
worst_states = c.find_worst_states("cases", 6, c.DEFAULT_DAYS)
states = [w[0] for w in worst_states]
for state in states:
    _ = c.plot_state(state, "cases")